In [1]:
import pandas as pd
import numpy as np
import io, os, sys
import glob


In [2]:
class data_clean():
    
    
    
    """
    centered the point clouds and make a total train/test set
    """
    def __init__(self, root_dir):
        """
        Args:
            files
            root_dir (string): Directory with the ModelNet.

        """
        self.root_dir = root_dir
    

    def centered_point_clouds(self, files):

        train_set = []
        
        for filename in files:
            train_airplane = pd.read_csv(filename, skiprows = [0,1],
                    header = None, names = ['x', 'y', 'z', 'c'], sep = " ")
            train_airplane = train_airplane[train_airplane.isnull().any(axis = 1)]
            train_airplane = train_airplane.iloc[:, :-1]
            train_airplane = train_airplane.to_numpy() 

            max_min = (np.abs(np.max(train_airplane, axis = 0)) - np.abs(np.min(train_airplane, axis = 0))) / 2


            Canonical_ST = np.zeros([1,3])

            Canonical_ST[0,0] = max_min[0]
            Canonical_ST[0,1] = max_min[1]
            Canonical_ST[0,2] = max_min[2]

            centered_point_clouds = train_airplane - Canonical_ST

            train_set.append(centered_point_clouds)

        return train_set


    def total_set(self):

        total_train_set = []
        total_test_set = []
        
        class_list = os.listdir(self.root_dir) # to find how many classes are, of course 40.....
        

        for i in range(len(class_list)):
            
            train_files = glob.glob(self.root_dir + class_list[i] + '/train/*.off')
            test_files = glob.glob(self.root_dir + class_list[i] + '/test/*.off')

            train = self.centered_point_clouds(train_files)
            test = self.centered_point_clouds(test_files)
            
            total_train_set.append(train)
            total_test_set.append(test)

        return total_train_set, total_test_set
    


In [3]:
clean_data = data_clean('./ModelNet40/')

In [4]:
total_train_set, total_test_set = clean_data.total_set()

In [5]:
np.save("total_train_set", total_train_set)
np.save("total_test_set", total_test_set)

In [6]:
total_train_set_test = np.load("total_train_set.npy", allow_pickle = True)
total_test_set_test = np.load("total_test_set.npy", allow_pickle = True)

In [10]:

print("how many classes: ", len(total_train_set_test), "and how many train set in the first class? " , len(total_train_set_test[0]))

how many classes:  40 and how many train set in the first class?  626
